In [1]:
# Import the necesssary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./cars images\N1.xml,1093,1396,645,727
1,./cars images\N100.xml,134,301,312,350
2,./cars images\N101.xml,31,139,128,161
3,./cars images\N102.xml,164,316,216,243
4,./cars images\N103.xml,813,1067,665,724


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./cars images\\N1.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./cars images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./cars images\\N1.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./cars images\\N1.jpeg',
 './cars images\\N100.jpeg',
 './cars images\\N101.jpeg',
 './cars images\\N102.jpeg',
 './cars images\\N103.jpeg',
 './cars images\\N104.jpeg',
 './cars images\\N105.jpeg',
 './cars images\\N106.jpeg',
 './cars images\\N107.jpeg',
 './cars images\\N108.jpeg',
 './cars images\\N109.jpeg',
 './cars images\\N11.jpeg',
 './cars images\\N110.jpeg',
 './cars images\\N111.jpeg',
 './cars images\\N112.jpeg',
 './cars images\\N113.jpeg',
 './cars images\\N114.jpeg',
 './cars images\\N116.jpeg',
 './cars images\\N117.jpeg',
 './cars images\\N118.jpeg',
 './cars images\\N119.jpeg',
 './cars images\\N12.jpeg',
 './cars images\\N120.jpeg',
 './cars images\\N121.jpeg',
 './cars images\\N122.jpeg',
 './cars images\\N123.jpeg',
 './cars images\\N124.jpeg',
 './cars images\\N126.jpeg',
 './cars images\\N127.jpeg',
 './cars images\\N128.jpeg',
 './cars images\\N129.jpeg',
 './cars images\\N130.jpeg',
 './cars images\\N131.jpeg',
 './cars images\\N132.jpeg',
 './cars images\\N

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./cars images\\N1.jpeg'

In [9]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [12]:
labels = df.iloc[:,1:].values

In [13]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [14]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [15]:
X.shape,y.shape

((283, 224, 224, 3), (283, 4))

In [16]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((226, 224, 224, 3), (57, 224, 224, 3), (226, 4), (57, 4))

### Deep Learning Model

In [17]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [18]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(512,activation="relu")(headmodel)
headmodel = Dense(256,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [19]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 111, 111, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 111, 111, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 111, 111, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 109, 109, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 109, 109, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 109, 109, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 109, 109, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 109, 109, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 109, 109, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 54, 54, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 54, 54, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 54, 54, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 54, 54, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 52, 52, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 74,130,404 (282.79 MB)

 Trainable params: 19,793,668 (75.51 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

block8_5_mixed (Concatenate)    (None, 5, 5, 448)    0           activation_179[0][0]             
                                                                 activation_182[0][0]             
__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, 5, 5, 2080)   933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, 5, 5, 2080)   0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, 5, 5, 2080)   0           block8_5[0][0]                   
__________________________________________________________________________________________________
conv2d_184

Total params: 73,663,490
Trainable params: 19,326,754
Non-trainable params: 54,336,736
__________________________________________________________________________________________________


### model training

In [20]:
from tensorflow.keras.callbacks import TensorBoard

In [21]:
tfb = TensorBoard('object_detection')

In [22]:
history = model.fit(x=x_train,y=y_train,batch_size=12,epochs=120,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 160s 6s/step - loss: 0.1323 - val_loss: 0.1418
Epoch 2/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - loss: 0.1197 - val_loss: 0.1089
Epoch 3/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - loss: 0.0958 - val_loss: 0.0976
Epoch 4/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - loss: 0.0930 - val_loss: 0.0906
Epoch 5/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 91s 5s/step - loss: 0.0855 - val_loss: 0.0863
Epoch 6/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - loss: 0.0786 - val_loss: 0.0861
Epoch 7/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 89s 5s/step - loss: 0.0746 - val_loss: 0.0810
Epoch 8/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 93s 5s/step - loss: 0.0734 - val_loss: 0.0847
Epoch 9/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 90s 5s/step - loss: 0.0776 - val_loss: 0.0818
Epoch 10/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 89s 5s/step - loss: 0.0709 - val_loss: 0.0809
Epoch 11/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 90s 5s/step - loss: 0.0757 - val_loss: 0.0846
Epoch 12/120
19/19 ━━━━━━━━━━━━━━━━━━━━ 94s 5s/step - loss: 0.

In [23]:
model.save('./models/object_detection.keras')